In [1]:
import os
import pandas as pd

if str(os.getcwdb()[-3:]).split("'")[1] != 'src':
    os.chdir(os.path.dirname(os.getcwdb()))

from utils.cleansing import *
from utils.modeling import *


In [17]:
df_diamonds = pd.read_csv(r'data\processed\diamonds_training.csv', index_col='id')
df_predict = pd.read_csv(r'data\processed\diamonds_testing.csv', index_col='id')


# Consideraciones
- Se localiza el tema en Kaggle: https://www.kaggle.com/datasets/shivam2503/diamonds
- Se empieza a trabajar con ese "dataset" (ver los archivos marcados como "UNUSED" y "no competition")
- Se detecta que existe una competición, si bien ya ha terminado: https://www.kaggle.com/competitions/diamonds-part-datamad0122/overview
- Se elige trabajar con los archivos de la competición, cuyas únicas diferencias es que hay un "train" y un "test", y que la variable "target" está escalada
- El "dataset" final es un listado de diamantes con sus características, y el objetivo es predecir el precio
- Se comparará lo obtenido con los resultados de la competición

# EDA
- Los pasos de esta primera parte se detallan de forma más pormenorizada, paso a paso, en el "notebook" titulado "EDA_diamonds"
- En ese "notebook" se hacen dos cosas:
1) Modificaciones esenciales; se liquidan duplicados, se cambia el nombre de las columnas y se pasan las categóricas a numéricas, tanto del "train" como del "test".

2) Modificaciones opcionales; se detectan y ponen a prueba las posibles modifiaciones que llevar a cabo con el "dataframe" de entrenamiento con tal de mejorar el resultado de los modelos. Los resortes de dichos cambios se guardan en forma de funciones (cuando son exclusivos de este proyecto) o clases (cuando es razonable guardarlos para análisis futuros), que se irán llamando a continuación según convenga.

# Modelaje
- Se importan los "dataframes" con las modificaciones esenciales
- Se van intercalando modificiaciones opcionales y diversos modelos hasta dar con el mejor resultado
- Los modelos se prueban en este "notebook" para mayor comodidad, pero se ejecutan sin detallarse en "train.py", desde donde se guardan en la carpeta "model"
- Estas son las modificaciones que se van intercalando:

---------- Cambios opcionales (probados) ---------- 

1) Borrado de "outliers" extremadamente altos ("depth (percentage)", "table (percentage)", "width (millimeters)", "depth (millimeters)").

2) Borrado de filas que tienen 0 en todas las variables de tamaño ("lenght (millimeters)", "width (millimeters)" y "depth (millimeters)").

3) Borrado de los "outliers" compartidos moderadamente altos ("depth (percentage)" y "table (percentage)").

4) Asignación del valor con 0 restante en "lenght" a partir del "width" correspondiente ("lenght (millimeters)").

5) Asignación del valor con 0 restante de "depth (millimeters)" a partir del "lenght", el "width" y el "depth (percentage)" correspondientes ("depth (millimeters)").

6) Uso del logaritmo ("weight (carat)", "lenght (millimeters)", "width (millimeters)" y "depth (millimeters)").

7) Asignación del "outlier" restante del "lenght" a partir del "width" correspondiente.

8) Imputación a valores máximos del "boxplot" ("weight (carat)").

9) Imputación a los valores máximos y mínimos del "boxplot" ("depth (percentage)" y "table (percentage)").

10) Neutralización de "outliers" con un modelo "ridge" ("depth (millimeters)").

11) Escalado general.

---------- Cambios apuntados (no probados) ----------

1) Sustitución de valores existentes por valores calculados ("depth (percentage)").

2) Borrado de las columnas con altísima correlación ("weight (carat)", "lenght (millimeters)", "width (millimeters)" y "depth (millimeters)").

3) Imputación de los valores máximos de "clarity quality" al que está un punto por debajo.


## Ronda 1: sin cambios
- Para la primera fase, se prueban todos los modelos sin hacer ninguna modificación adicional
- En esta primera ronda están más detallados los usos de la clase "Regression", que hereda de "Model", para que sirva como ejemplo
- Como es de esperar, los resultados no son muy buenos
- Pueden compararse con los de la competición de Kaggle: https://www.kaggle.com/competitions/diamonds-part-datamad0122/leaderboard

In [3]:
# Lo primero es decirle a la clase con qué modelos se va a trabajar a lo largo de todo el proceso
Regression.add_models(['LinearRegression',
                        'Ridge',
                        'DecisionTreeRegressor',
                        'KNeighborsRegressor',
                        'RandomForestRegressor',
                        'SVR'
                        ]
                        )


In [4]:
# Se crea la instancia de la clase "Regression" con la columna "price" como "target"
round_1 = Regression(df_diamonds, 'price')


In [5]:
# Se separa el "dataframe" con los parámetros por defecto. Se guardan las porciones por si acaso
X_train, X_test, y_train, y_test = round_1.split_dataframe()


In [6]:
# Se solicitan 10 "folds", del cual se usará el mejor para comparar los modelos y ver cuál llega más lejos
# Se coge el mejor, y no la media de las métricas, porque de 10 cortes es probable que salgan números de "cortes malos" dispares para los diferentes modelos
# Si fuera el caso, la comparación con las medias seria injusta
# En cambio, con el mejor, es muy probable que de 10 cortes al menos uno de ellos saque el máximo partido a las métricas para cada modelo
# Así, se comparan en mayor igualdad de condiciones
# Como la "target" es de regresión, la instancia seleccionará automáticamente "KFold" en lugar de "StratifiedFold"
# Se establece un "random_state" para los modelos que lo requieren, que siempre será el mismo
round_1_dict = round_1.apply_models(params_list=[['DecisionTreeRegressor', 'random_state=43'],
                                                    ['RandomForestRegressor', 'random_state=43']
                                                ],
                                    kfolds_num=10
                                    )


-- Regression: using best of 10 folds --
Starting LinearRegression:
- LinearRegression done in 0.39 sec(s). Total time: 0.39
Starting Ridge:
- Ridge done in 0.25 sec(s). Total time: 0.64
Starting KNeighborsRegressor:
- KNeighborsRegressor done in 6.62 sec(s). Total time: 7.25
Starting SVR:
- SVR done in 375.04 sec(s). Total time: 382.29
Starting DecisionTreeRegressor: random_state=43:
- DecisionTreeRegressor: random_state=43 done in 2.69 sec(s). Total time: 384.98
Starting RandomForestRegressor: random_state=43:
- RandomForestRegressor: random_state=43 done in 320.79 sec(s). Total time: 705.77


In [7]:
# Los resultados, así como el modelo entrenado, pueden visualizarse en un diccionario
round_1_dict


In [8]:
# Acto seguido, se miran las métricas
round_1_metrics = round_1.evaluate_metrics()

round_1_metrics


{'LinearRegression': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.1751789 , 8.94534424, 7.94609508, ..., 9.13303572, 8.78005708,
         8.22465851]),
  'model': LinearRegression(),
  'metrics': {'rmse': 0.222037823248969,
   'mse': 0.0493007949531404,
   'mae': 0.12289980638635319,
   'r2_score': 0.9529090797126748,
   'mape': 0.01587291534328809}},
 'Ridge': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.17485364, 8.94499768, 7.94593798, ..., 9.13327901, 8.78003269,
         8.22426296]),
  'model': Ridge(),
  'metrics': {'rmse': 0.2219231703319744,
   'mse': 0.04924989353019452,
   'mae': 0.1229616970889257,
   'r2_score': 0.9529576995138916,
   'mape': 0.015880870501013333}},
 'KNeighborsRegressor': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.2228, 9.072 , 8.0682, ..., 9.09  , 8.7678, 8.706 ]),
  'model': KNeighborsRegressor(),
  'metrics': {'rmse':

In [9]:
# Para una mejor visualización, se ponen en un "dataframe"
# Las predicciones no son muy buenas, si bien el r2_score es alto para todos los casos
round_1.create_dataframe()


,LinearRegression,Ridge,KNeighborsRegressor,SVR,DecisionTreeRegressor: random_state=43,RandomForestRegressor: random_state=43,BEST,WORST
rmse,0.222038,0.221923,0.183865,0.209064,0.130142,0.098101,RandomForestRegressor: random_state=43,LinearRegression
mse,0.049301,0.049250,0.033806,0.043708,0.016937,0.009624,RandomForestRegressor: random_state=43,LinearRegression
mae,0.122900,0.122962,0.135156,0.127477,0.088698,0.067123,KNeighborsRegressor,KNeighborsRegressor
r2_score,0.952909,0.952958,0.967709,0.958252,0.983822,0.990808,RandomForestRegressor: random_state=43,LinearRegression
mape,0.015873,0.015881,0.017806,0.016523,0.011417,0.008684,KNeighborsRegressor,KNeighborsRegressor


## Ronda 2: escalado
- Se repite la ronda 1, pero esta vez se escalan las variables
- En esta segunda ronda, así como en las siguientes donde se emplee con otros atributos, están más detallados los usos de la clase "Cleansing", para que sirva como ejemplo
- Queda probado que el escalado "MinMax" mejora los resultados

In [22]:
# Se crea una instancia de la clase "Cleansing" con los dos "dataframes", el de datos históricos y el de la predicción
df_diamonds_2 = df_diamonds.copy()
df_predict_2 = df_predict.copy()

cleansing_2 =  Cleansing([df_diamonds_2, df_predict_2], 'price')


In [23]:
# Se aplica el escalado "MinMax"
cleansing_2.apply_scalar('minmax')

df_diamonds_2.head()


Minmax scaler completed


,weight (carat),cut quality,color quality,clarity quality,depth (percentage),table (percentage),lenght (millimeters),width (millimeters),depth (millimeters),price
id,,,,,,,,,,
0,0.020790,0.75,1.000000,0.142857,0.538889,0.274510,0.401304,0.072666,0.084277,0.139581
1,0.168399,1.00,0.833333,0.714286,0.547222,0.235294,0.597765,0.109677,0.127044,0.837485
2,0.108108,1.00,0.666667,0.428571,0.522222,0.294118,0.531657,0.097453,0.111321,0.541554
3,0.182952,0.50,0.500000,0.142857,0.561111,0.254902,0.608939,0.110357,0.129560,0.637238
4,0.033264,0.75,0.500000,0.571429,0.536111,0.294118,0.418994,0.077250,0.088679,0.197534


In [24]:
# Se pone a prueba con el mismo proceso que en la ronda anterior
# Es mucho más rápido
round_2 = Regression(df_diamonds_2, 'price')
round_2.split_dataframe()
round_2_dict = round_2.apply_models(params_list=[['DecisionTreeRegressor', 'random_state=43'],
                                                    ['RandomForestRegressor', 'random_state=43']
                                                ],
                                    kfolds_num=10
                                    )
round_2.evaluate_metrics()
round_2.create_dataframe()


-- Regression: using best of 10 folds --
Starting LinearRegression:
- LinearRegression done in 0.3 sec(s). Total time: 0.3
Starting Ridge:
- Ridge done in 0.15 sec(s). Total time: 0.44
Starting KNeighborsRegressor:
- KNeighborsRegressor done in 3.27 sec(s). Total time: 3.72
Starting SVR:
- SVR done in 9.67 sec(s). Total time: 13.38
Starting DecisionTreeRegressor: random_state=43:
- DecisionTreeRegressor: random_state=43 done in 2.96 sec(s). Total time: 16.34
Starting RandomForestRegressor: random_state=43:
- RandomForestRegressor: random_state=43 done in 263.59 sec(s). Total time: 279.94


,LinearRegression,Ridge,KNeighborsRegressor,SVR,DecisionTreeRegressor: random_state=43,RandomForestRegressor: random_state=43,BEST,WORST
rmse,0.054757,0.053697,0.038099,0.038442,0.032114,0.024211,RandomForestRegressor: random_state=43,LinearRegression
mse,0.002998,0.002883,0.001452,0.001478,0.001031,0.000586,RandomForestRegressor: random_state=43,LinearRegression
mae,0.030308,0.031563,0.026454,0.029648,0.021970,0.016586,Ridge,Ridge
r2_score,0.952909,0.954715,0.977203,0.976790,0.983803,0.990794,RandomForestRegressor: random_state=43,LinearRegression
mape,0.093012,0.099544,0.096992,0.118786,0.066269,0.052825,SVR,SVR


## Ronda 3: borrado de valores
- Se borran todos los "outliers" extremadamente altos ("depth (percentage)", "table (percentage)", "width (millimeters)", "depth (millimeters)")
- Se borran las filas que tienen 0 en estas tres columnas: "lenght (millimeters)", "width (millimeters)" y "depth (millimeters)"
- Se borran los "outliers" moderadamente altos compartidos de "depth (percentage)" y "table (percentage)"

In [25]:
# Se crea la instancia
df_diamonds_3 = df_diamonds.copy()
df_predict_3 = df_predict.copy()

cleansing_3 =  Cleansing([df_diamonds_2, df_predict_2], 'price')


In [ ]:
# Se aplican los borrados
cleansing_2.remove_elements(, condition, number)

df_diamonds_2.head()
